In [1]:
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
from citipy import citipy
import math
import time

# Google API Key
from config import gkey
# OpenWeatherMap API key
from api_keys import api_key

from sqlalchemy import create_engine

import pymysql
pymysql.install_as_MySQLdb()

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

### Store CSV into DataFrame: Vulture Migration Paths

In [2]:
excel_file = "Resources/Turkey vultures in North and South America - migration.xlsx"
vulture_data_df = pd.read_excel(excel_file, index_col=None)
vulture_data_df.head(2)

,event-id,visible,timestamp,location-long,location-lat,manually-marked-outlier,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,...,animal-life-stage,animal-mass,attachment-type,deployment-comments,deployment-id,duty-cycle,study-site,tag-manufacturer-name,tag-mass,tag-model
0,283203879,True,2003-11-14 16:00:00.000,-75.39717,40.48933,False,gps,Cathartes aura,42500,Butterball,...,adult,2372.0,harness,trapped in Pennsylvania using padded-leg hold ...,42500-Butterball,1 fix per hour,East Coast of North America,Microwave Telemetry,70,PTT100
1,283203880,True,2003-11-14 17:00:00.000,-75.39717,40.48933,False,gps,Cathartes aura,42500,Butterball,...,adult,2372.0,harness,trapped in Pennsylvania using padded-leg hold ...,42500-Butterball,1 fix per hour,East Coast of North America,Microwave Telemetry,70,PTT100


In [3]:
vulture_data_df.columns

Index(['event-id', 'visible', 'timestamp', 'location-long', 'location-lat',
       'manually-marked-outlier', 'sensor-type',
       'individual-taxon-canonical-name', 'tag-local-identifier',
       'individual-local-identifier', 'study-name', 'utm-easting',
       'utm-northing', 'utm-zone', 'study-timezone', 'study-local-timestamp',
       'tag-id', 'animal-id', 'animal-taxon', 'deploy-on-date',
       'deploy-off-date', 'animal-comments', 'animal-life-stage',
       'animal-mass', 'attachment-type', 'deployment-comments',
       'deployment-id', 'duty-cycle', 'study-site', 'tag-manufacturer-name',
       'tag-mass', 'tag-model'],
      dtype='object')

In [4]:
vulture_data_df.keys()

Index(['event-id', 'visible', 'timestamp', 'location-long', 'location-lat',
       'manually-marked-outlier', 'sensor-type',
       'individual-taxon-canonical-name', 'tag-local-identifier',
       'individual-local-identifier', 'study-name', 'utm-easting',
       'utm-northing', 'utm-zone', 'study-timezone', 'study-local-timestamp',
       'tag-id', 'animal-id', 'animal-taxon', 'deploy-on-date',
       'deploy-off-date', 'animal-comments', 'animal-life-stage',
       'animal-mass', 'attachment-type', 'deployment-comments',
       'deployment-id', 'duty-cycle', 'study-site', 'tag-manufacturer-name',
       'tag-mass', 'tag-model'],
      dtype='object')

### Create new data with select columns

In [5]:
# Select columns 
new_vulture_data_df = vulture_data_df[['event-id', 'timestamp', 'location-long', 'location-lat','individual-taxon-canonical-name', 'tag-local-identifier',
       'animal-id']].copy()
new_vulture_data_df.head()

,event-id,timestamp,location-long,location-lat,individual-taxon-canonical-name,tag-local-identifier,animal-id
0,283203879,2003-11-14 16:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
1,283203880,2003-11-14 17:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
2,283203881,2003-11-14 18:00:00.000,-75.33317,40.32467,Cathartes aura,42500,Butterball
3,283203882,2003-11-14 19:00:00.000,-75.35617,40.33983,Cathartes aura,42500,Butterball
4,283203883,2003-11-14 20:00:00.000,-75.42650,40.31550,Cathartes aura,42500,Butterball


### Clean DataFrame

In [6]:
new_vulture_data_df.count()

event-id                           220077
timestamp                          220077
location-long                      220077
location-lat                       220077
individual-taxon-canonical-name    220077
tag-local-identifier               220077
animal-id                          220077
dtype: int64

In [7]:
# drop rows without long and lat
new_vulture_data_df = new_vulture_data_df.dropna(how="any")
new_vulture_data_df.count()

event-id                           220077
timestamp                          220077
location-long                      220077
location-lat                       220077
individual-taxon-canonical-name    220077
tag-local-identifier               220077
animal-id                          220077
dtype: int64

In [8]:
# filter data to only keep turkey vulture (Cathartes aura) data
new_vulture_data_df = new_vulture_data_df.loc[new_vulture_data_df
                                              ["individual-taxon-canonical-name"] == "Cathartes aura", :]
new_vulture_data_df.count()

event-id                           220077
timestamp                          220077
location-long                      220077
location-lat                       220077
individual-taxon-canonical-name    220077
tag-local-identifier               220077
animal-id                          220077
dtype: int64

In [9]:
new_vulture_data_df.dtypes

event-id                             int64
timestamp                           object
location-long                      float64
location-lat                       float64
individual-taxon-canonical-name     object
tag-local-identifier                 int64
animal-id                           object
dtype: object

In [10]:
# check to prevent duplicate loading (inqury for database)

### Connect to local database

In [11]:
database_path = "vulture_etl"
engine = create_engine(f"sqlite:///{database_path}")

### Check for tables

In [12]:
engine.table_names()

['city_data',
 'city_df',
 'counts_per_bird',
 'migration_join',
 'migration_paths',
 'vulture_detail']

### Use pandas to load csv converted DataFrame into database

In [13]:
new_vulture_data_df.to_sql(name='migration_paths', con=engine, if_exists='replace', index=False)

### Use pandas to import second csv: Vulture Info by name

In [14]:
csv_file = "Resources/Turkey vultures in North and South America-reference-data.csv"
vulture_info_df = pd.read_csv(csv_file, low_memory=False)
vulture_info_df.head()

,tag-id,animal-id,animal-taxon,deploy-on-date,deploy-off-date,animal-comments,animal-life-stage,animal-mass,attachment-type,deployment-comments,deployment-id,duty-cycle,study-site,tag-manufacturer-name,tag-mass,tag-model
0,42500,Butterball,Cathartes aura,2003-11-14 16:00:00.000,2004-03-14 20:00:01.000,migratory,adult,2372.0,harness,trapped in Pennsylvania using padded-leg hold ...,42500-Butterball,1 fix per hour,East Coast of North America,Microwave Telemetry,70.0,PTT100
1,52067,Irma,Cathartes aura,2004-09-06 17:00:00.000,2013-03-18 22:00:01.000,non-migratory,adult,2012.0,harness,trapped in Pennsylvania using padded-leg hold ...,52067-Irma,1 fix per hour,East Coast of North America,Microwave Telemetry,70.0,PTT100
2,42500,Schaumboch,Cathartes aura,2004-10-08 15:00:00.000,2006-03-29 17:00:01.000,migratory,adult,1951.0,harness,trapped in Pennsylvania using padded-leg hold ...,42500-Schaumboch,1 fix per hour,East Coast of North America,Microwave Telemetry,70.0,PTT100
3,52069,Disney,Cathartes aura,2004-10-11 14:00:00.000,2011-10-18 23:00:01.000,migratory,adult,2108.0,harness,trapped in Pennsylvania using padded-leg hold ...,52069-Disney,1 fix per hour,East Coast of North America,Microwave Telemetry,70.0,PTT100
4,57954,Prado,Cathartes aura,2005-11-02 15:00:00.000,2009-07-07 00:00:01.000,non-migratory,adult,1710.0,harness,trapped in California using walk-in traps,57954-Prado,1 fix per hour,West Coast of North America,Microwave Telemetry,70.0,PTT100


In [15]:
vulture_info_df.columns

Index(['tag-id', 'animal-id', 'animal-taxon', 'deploy-on-date',
       'deploy-off-date', 'animal-comments', 'animal-life-stage',
       'animal-mass', 'attachment-type', 'deployment-comments',
       'deployment-id', 'duty-cycle', 'study-site', 'tag-manufacturer-name',
       'tag-mass', 'tag-model'],
      dtype='object')

In [16]:
# Select columns 
new_vulture_info_df = vulture_info_df[['tag-id', 'animal-id', 'animal-taxon', 'deploy-on-date',
       'deploy-off-date', 'animal-comments', 'animal-life-stage',
       'animal-mass',  'deployment-comments',
        'study-site']].copy()
new_vulture_info_df.head(1)

,tag-id,animal-id,animal-taxon,deploy-on-date,deploy-off-date,animal-comments,animal-life-stage,animal-mass,deployment-comments,study-site
0,42500,Butterball,Cathartes aura,2003-11-14 16:00:00.000,2004-03-14 20:00:01.000,migratory,adult,2372.0,trapped in Pennsylvania using padded-leg hold ...,East Coast of North America


In [17]:
new_vulture_info_df.count()

tag-id                 19
animal-id              19
animal-taxon           19
deploy-on-date         19
deploy-off-date        19
animal-comments        19
animal-life-stage      19
animal-mass            12
deployment-comments    19
study-site             19
dtype: int64

In [18]:
new_vulture_info_df.to_sql(name='vulture_detail', con=engine, if_exists='replace', index=False)

### Confirm data has been added by querying
* NOTE: can also check using pgAdmin

In [19]:
pd.read_sql_query('select * from migration_paths', con=engine).head()

,event-id,timestamp,location-long,location-lat,individual-taxon-canonical-name,tag-local-identifier,animal-id
0,283203879,2003-11-14 16:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
1,283203880,2003-11-14 17:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
2,283203881,2003-11-14 18:00:00.000,-75.33317,40.32467,Cathartes aura,42500,Butterball
3,283203882,2003-11-14 19:00:00.000,-75.35617,40.33983,Cathartes aura,42500,Butterball
4,283203883,2003-11-14 20:00:00.000,-75.42650,40.31550,Cathartes aura,42500,Butterball


In [20]:
pd.read_sql_query('select * from vulture_detail', con=engine).head()

,tag-id,animal-id,animal-taxon,deploy-on-date,deploy-off-date,animal-comments,animal-life-stage,animal-mass,deployment-comments,study-site
0,42500,Butterball,Cathartes aura,2003-11-14 16:00:00.000,2004-03-14 20:00:01.000,migratory,adult,2372.0,trapped in Pennsylvania using padded-leg hold ...,East Coast of North America
1,52067,Irma,Cathartes aura,2004-09-06 17:00:00.000,2013-03-18 22:00:01.000,non-migratory,adult,2012.0,trapped in Pennsylvania using padded-leg hold ...,East Coast of North America
2,42500,Schaumboch,Cathartes aura,2004-10-08 15:00:00.000,2006-03-29 17:00:01.000,migratory,adult,1951.0,trapped in Pennsylvania using padded-leg hold ...,East Coast of North America
3,52069,Disney,Cathartes aura,2004-10-11 14:00:00.000,2011-10-18 23:00:01.000,migratory,adult,2108.0,trapped in Pennsylvania using padded-leg hold ...,East Coast of North America
4,57954,Prado,Cathartes aura,2005-11-02 15:00:00.000,2009-07-07 00:00:01.000,non-migratory,adult,1710.0,trapped in California using walk-in traps,West Coast of North America


In [21]:
engine.table_names()

['city_data',
 'city_df',
 'counts_per_bird',
 'migration_join',
 'migration_paths',
 'vulture_detail']

In [22]:
conn = engine.connect()

from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

session = Session(bind=engine)
Base.metadata.create_all(engine)
session.commit()

In [23]:
new_vulture_data_df.head()

,event-id,timestamp,location-long,location-lat,individual-taxon-canonical-name,tag-local-identifier,animal-id
0,283203879,2003-11-14 16:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
1,283203880,2003-11-14 17:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
2,283203881,2003-11-14 18:00:00.000,-75.33317,40.32467,Cathartes aura,42500,Butterball
3,283203882,2003-11-14 19:00:00.000,-75.35617,40.33983,Cathartes aura,42500,Butterball
4,283203883,2003-11-14 20:00:00.000,-75.42650,40.31550,Cathartes aura,42500,Butterball


In [24]:
vulture_city = new_vulture_data_df[['location-lat', 'location-long']]
vulture_city.head()

,location-lat,location-long
0,40.48933,-75.39717
1,40.48933,-75.39717
2,40.32467,-75.33317
3,40.33983,-75.35617
4,40.31550,-75.42650


# list of cities

In [25]:
lats = new_vulture_data_df['location-lat'].values.tolist()
lngs = new_vulture_data_df['location-long'].values.tolist()
type(lngs)

list

In [26]:
lat_lngs = []
cities = []
lat_lngs = zip(lats, lngs)
lat_lngs_s = set(lat_lngs)
lat_lngs_a = list(lat_lngs_s)
print(lat_lngs_a[0])

(31.75567, -82.64317)


In [27]:
ll_arraycut = lat_lngs_a[::500]
print(ll_arraycut[0])
print(ll_arraycut[0][0])

(31.75567, -82.64317)
31.75567


In [28]:
add_array = []
for i in range(200):
    target_lat = ll_arraycut[i][0]
    target_lng = ll_arraycut[i][1]
    base_url = ('https://maps.googleapis.com/maps/api/geocode/json?latlng={0},{1}&key={2}').format(target_lat, target_lng, gkey)
    geo_data = requests.get(base_url).json()
    
    try:
        res = geo_data["plus_code"]["compound_code"]
        res_arr = res.split(", ")
        add_array.append(res_arr)
    except (KeyError, IndexError):
        pprint("Could not find information")

city_arr = pd.DataFrame(add_array) 
city_arr[0] = city_arr[0].map(lambda x: str(x)[8:])
city_arr = city_arr.rename(columns={0: 'cities'})
# city_array = new_array.loc[:, '1':'3']
print(city_arr.head(20))

'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
                      cities                               1          2     3
0                     Denton                              GA        USA  None
1                  Weatherly                              PA        USA  None
2                       Fame                              WV        USA  None
3                   Leoville                              SK     Canada  None
4                  Bethlehem                              PA        USA  None
5                    Anaheim                              CA        USA  None
6               Spring Creek                              SD        USA  None
7                     Zigler                              WV        USA  None
8               Ataliva Roca                        La Pampa  Arge

In [29]:
city_arr['Max Temp'] = ""
# print(city_arr)
city_df = city_arr[["cities", "Max Temp"]]
print(city_df.head(20))

                      cities Max Temp
0                     Denton         
1                  Weatherly         
2                       Fame         
3                   Leoville         
4                  Bethlehem         
5                    Anaheim         
6               Spring Creek         
7                     Zigler         
8               Ataliva Roca         
9             East Cathlamet         
10                    Bamepa         
11                  Leoville         
12                    Borden         
13                   Glendon         
14                  Leoville         
15  Porto Quinze de Novembro         
16                   Numidia         
17                Hinesville         
18               Cherry Hill         
19                     Choro         


In [30]:
limit = len(city_df)/50
set_var = 1
data_count = 50
dummy_array = [];

base_url = "http://api.openweathermap.org/data/2.5/weather?"
params = {'appid': api_key, 'units': 'Imperial'}

for index, row in city_df.iterrows():
    if len(dummy_array) < data_count and set_var <= math.ceil(limit):
        city_row = row["cities"]
        dummy_array.append(city_row)
       
        params['q'] = city_row

        response = requests.get(base_url, params=params).json()
    
    else:
        set_var = set_var + 1
        data_count = data_count + 49
        time.sleep(60)

    try:
        city_df.loc[index, 'Max Temp'] = response['main']['temp_max']
        
    except (KeyError, IndexError):
        pprint("Could not find information")

'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not find information'
'Could not fin

In [31]:
city_df.head(30)

,cities,Max Temp
0,Denton,79
1,Weatherly,53.01
2,Fame,73.4
3,Leoville,50.71
4,Bethlehem,54
5,Anaheim,64.4
6,Spring Creek,61
7,Zigler,53.6
8,Ataliva Roca,
9,East Cathlamet,53.6


In [32]:

city_df.describe()

,cities,Max Temp
count,192,192
unique,118,50
top,Leoville,
freq,10,58


In [33]:
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    cities.append(city)

In [34]:
len(cities)
cities[-1]

'pottsville'

In [35]:
cities_citypy = new_vulture_data_df[['event-id', 'location-lat', 'location-long']]
cities_citypy["cities"] = pd.DataFrame({"cities" : cities})
cities_citypy = cities_citypy[['event-id', 'location-lat', 'location-long', 'cities']]
cities_citypy.head(30)

,event-id,location-lat,location-long,cities
0,283203879,40.48933,-75.39717,emmaus
1,283203880,40.48933,-75.39717,emmaus
2,283203881,40.32467,-75.33317,lansdale
3,283203882,40.33983,-75.35617,lansdale
4,283203883,40.31550,-75.42650,lansdale
5,283203884,40.30383,-75.40500,lansdale
6,283203885,40.29567,-75.41133,lansdale
7,283203886,40.29567,-75.41167,lansdale
8,283203887,40.29600,-75.41183,lansdale
9,283203888,40.29583,-75.41167,lansdale


In [36]:
cities_citypy.to_sql(name='city_data', con=engine, if_exists='replace', index=False)

In [37]:
migration_join = pd.merge(new_vulture_data_df, new_vulture_info_df,  how='left', on='animal-id')
migration_join = migration_join.drop(axis = 1, columns=['individual-taxon-canonical-name','tag-id', 
                                                        'deploy-on-date', 'deploy-off-date', 'deployment-comments',
                                                       'study-site'])
migration_join.head(10)


,event-id,timestamp,location-long,location-lat,tag-local-identifier,animal-id,animal-taxon,animal-comments,animal-life-stage,animal-mass
0,283203879,2003-11-14 16:00:00.000,-75.39717,40.48933,42500,Butterball,Cathartes aura,migratory,adult,2372.0
1,283203880,2003-11-14 17:00:00.000,-75.39717,40.48933,42500,Butterball,Cathartes aura,migratory,adult,2372.0
2,283203881,2003-11-14 18:00:00.000,-75.33317,40.32467,42500,Butterball,Cathartes aura,migratory,adult,2372.0
3,283203882,2003-11-14 19:00:00.000,-75.35617,40.33983,42500,Butterball,Cathartes aura,migratory,adult,2372.0
4,283203883,2003-11-14 20:00:00.000,-75.42650,40.31550,42500,Butterball,Cathartes aura,migratory,adult,2372.0
5,283203884,2003-11-14 21:00:00.000,-75.40500,40.30383,42500,Butterball,Cathartes aura,migratory,adult,2372.0
6,283203885,2003-11-14 22:00:00.000,-75.41133,40.29567,42500,Butterball,Cathartes aura,migratory,adult,2372.0
7,283203886,2003-11-14 23:00:00.000,-75.41167,40.29567,42500,Butterball,Cathartes aura,migratory,adult,2372.0
8,283203887,2003-11-15 00:00:00.000,-75.41183,40.29600,42500,Butterball,Cathartes aura,migratory,adult,2372.0
9,283203888,2003-11-15 01:00:00.000,-75.41167,40.29583,42500,Butterball,Cathartes aura,migratory,adult,2372.0


In [38]:
counts_per_bird = migration_join.groupby('animal-id').count()['event-id']
counts_per_bird

animal-id
Argentina        4058
Butterball       4358
Disney          28578
Domingo          3235
Irma            18314
La Pampa         4032
Leo             32947
Mac              7864
Mark             5353
Mary             3299
Morongo         19868
Prado           20967
Rosalie         28101
Sarkis           8451
Schaumboch       4358
Steamhouse 1     5655
Steamhouse 2     9416
Whitey           3565
Young Luro       7658
Name: event-id, dtype: int64

In [39]:
migration_join.groupby('animal-id').count()['event-id']

animal-id
Argentina        4058
Butterball       4358
Disney          28578
Domingo          3235
Irma            18314
La Pampa         4032
Leo             32947
Mac              7864
Mark             5353
Mary             3299
Morongo         19868
Prado           20967
Rosalie         28101
Sarkis           8451
Schaumboch       4358
Steamhouse 1     5655
Steamhouse 2     9416
Whitey           3565
Young Luro       7658
Name: event-id, dtype: int64

In [40]:
migration_join.describe()

,event-id,location-long,location-lat,tag-local-identifier,animal-mass
count,2.200770e+05,220077.000000,220077.000000,220077.000000,156718.000000
mean,2.831478e+08,-92.481423,27.992753,61601.288554,1948.104864
std,6.220656e+04,21.340739,22.163269,11580.548715,202.827831
min,2.830388e+08,-123.925500,-39.821400,42500.000000,1600.000000
25%,2.830938e+08,-111.420000,17.959670,52069.000000,1750.000000
50%,2.831488e+08,-82.562330,33.871830,57956.000000,1975.000000
75%,2.832038e+08,-75.193670,40.961500,65545.000000,2108.000000
max,2.832545e+08,-48.578800,54.018170,85756.000000,2431.000000


In [41]:
migration_join.to_sql(name='migration_join', con=engine, if_exists='replace', index=False)
counts_per_bird.to_sql(name='counts_per_bird', con=engine, if_exists='replace', index=False)

In [42]:
session.commit()

In [43]:
engine.table_names()

['city_data',
 'city_df',
 'counts_per_bird',
 'migration_join',
 'migration_paths',
 'vulture_detail']